# Exploring Ranking Models in Information Retrieval

## Objective
Understand the practical implementation and differences between the Vector Space Model and the Binary Independence Model in ranking documents relative to a user query.

### Step 1: Data Preprocessing

Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.
Write a function to load and preprocess the text documents from a specified directory. This step involves reading each file, converting the text to lowercase for uniform processing, and storing the results in a dictionary.

In [1]:
import os
import re
import collections
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np

In [2]:
# Define the path to the directory containing the text files
CORPUS_DIR = "../libros"
documents = {}

In [3]:
def clean_text(text):
    # Remover símbolos y paréntesis utilizando expresiones regulares
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

for filename in os.listdir(CORPUS_DIR):
    if filename.endswith('.txt'):
        file_path = os.path.join(CORPUS_DIR, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read().lower()
            cleaned_text = clean_text(text)
            documents[filename] = cleaned_text

In [4]:
# Dictionary to save the doc normalized
normalized_word_counts = {}

In [5]:
for doc in documents:
    word_count = collections.Counter(documents[doc].split())
    total_words = sum(word_count.values())
    normalized_word_count = {word: count/total_words for word, count in word_count.items()}
    normalized_word_counts[doc] = normalized_word_count

In [6]:
df = pd.DataFrame.from_dict(normalized_word_counts)

In [7]:
df_no_nan = df.fillna(0)
df_no_nan = df_no_nan.rename_axis('Files')
df_no_nan.head(10)

,pg100.txt,pg10676.txt,pg10681.txt,pg1080.txt,pg11.txt,pg1184.txt,pg120.txt,pg1232.txt,pg1259.txt,pg1260.txt,...,pg73447.txt,pg7370.txt,pg74.txt,pg76.txt,pg768.txt,pg84.txt,pg844.txt,pg8800.txt,pg98.txt,pg996.txt
Files,,,,,,,,,,,,,,,,,,,,,
the,0.031548,0.070610,0.024495,0.055297,0.061879,0.061475,0.064093,0.058870,0.059102,0.042232,...,0.088325,0.061040,0.053341,0.044282,0.039849,0.056090,0.033986,0.051372,0.059054,0.052207
project,0.000103,0.000965,0.000492,0.013824,0.002984,0.000233,0.001234,0.001662,0.000400,0.000478,...,0.002612,0.001479,0.001192,0.000780,0.000765,0.001127,0.003720,0.000788,0.000634,0.000221
gutenberg,0.000090,0.000745,0.000428,0.013514,0.002950,0.000190,0.001220,0.001643,0.000363,0.000462,...,0.002392,0.001462,0.001178,0.000762,0.000732,0.001114,0.003678,0.000779,0.000626,0.000205
ebook,0.000013,0.000110,0.000069,0.002019,0.000441,0.000028,0.000182,0.000246,0.000073,0.000069,...,0.000357,0.000218,0.000176,0.000114,0.000109,0.000166,0.000550,0.000116,0.000094,0.000033
of,0.019516,0.030938,0.019502,0.040230,0.021395,0.027768,0.025306,0.034317,0.025328,0.023714,...,0.051983,0.044183,0.021496,0.015634,0.019697,0.035357,0.021939,0.023071,0.029765,0.031301
complete,0.000025,0.000059,0.000138,0.000000,0.000000,0.000086,0.000084,0.000094,0.000041,0.000101,...,0.000055,0.000017,0.000149,0.000000,0.000050,0.000102,0.000042,0.000063,0.000043,0.000056
works,0.000069,0.000355,0.000192,0.004970,0.001119,0.000082,0.000449,0.000793,0.000147,0.000196,...,0.000935,0.000588,0.000433,0.000315,0.000269,0.000512,0.001353,0.000394,0.000245,0.000139
william,0.000092,0.000000,0.000000,0.000000,0.000170,0.000000,0.000000,0.000000,0.000004,0.000000,...,0.000330,0.000034,0.000000,0.000131,0.000000,0.000320,0.000000,0.000027,0.000000,0.000000
shakespeare,0.000009,0.000000,0.000015,0.000000,0.000034,0.000006,0.000000,0.000000,0.000004,0.000000,...,0.000000,0.000000,0.000000,0.000018,0.000000,0.000013,0.000000,0.000000,0.000000,0.000014


In [8]:
df_no_nan.to_csv("vect_normalized.csv",index=True)

### Step 2:  Vector Space Model (VSM)

Task: Implement a simple Vector Space Model using term frequency.

Requirements:
* _Document and Query Representation:_ Convert each document and the query into a vector where each dimension corresponds to a term from the corpus. Use simple term frequency for weighting.
* _Cosine Similarity Calculation:_ Calculate the cosine similarity between the query vector and each document vector.
* _Ranking:_ Rank the documents based on their cosine similarity scores from highest to lowest.

In [9]:
df = pd.read_csv("vect_normalized.csv")

In [10]:
import math
def euclidean_distance(vector1, vector2):
    squared_diff = sum([(x - y)**2 for x, y in zip(vector1, vector2)])
    distance = math.sqrt(squared_diff)
    return distance

In [11]:
def get_query_vector(query):
    query_vector = df.loc[df['Files'] == query].values[0][1:]
    return query_vector

In [12]:
def rank_documents(query):
    query_vector = get_query_vector(query)
    similarities = []

    for i in range(df.shape[1] - 1):
        doc_vector = df.iloc[:, i + 1].values
        distance = euclidean_distance(query_vector, doc_vector)
        similarities.append((df.columns[i + 1], distance))

    ranked_documents = sorted(similarities, key=lambda x: x[1])
    return ranked_documents

In [13]:
df.head(10)

,Files,pg100.txt,pg10676.txt,pg10681.txt,pg1080.txt,pg11.txt,pg1184.txt,pg120.txt,pg1232.txt,pg1259.txt,...,pg73447.txt,pg7370.txt,pg74.txt,pg76.txt,pg768.txt,pg84.txt,pg844.txt,pg8800.txt,pg98.txt,pg996.txt
0,the,0.031548,0.070610,0.024495,0.055297,0.061879,0.061475,0.064093,0.058870,0.059102,...,0.088325,0.061040,0.053341,0.044282,0.039849,0.056090,0.033986,0.051372,0.059054,0.052207
1,project,0.000103,0.000965,0.000492,0.013824,0.002984,0.000233,0.001234,0.001662,0.000400,...,0.002612,0.001479,0.001192,0.000780,0.000765,0.001127,0.003720,0.000788,0.000634,0.000221
2,gutenberg,0.000090,0.000745,0.000428,0.013514,0.002950,0.000190,0.001220,0.001643,0.000363,...,0.002392,0.001462,0.001178,0.000762,0.000732,0.001114,0.003678,0.000779,0.000626,0.000205
3,ebook,0.000013,0.000110,0.000069,0.002019,0.000441,0.000028,0.000182,0.000246,0.000073,...,0.000357,0.000218,0.000176,0.000114,0.000109,0.000166,0.000550,0.000116,0.000094,0.000033
4,of,0.019516,0.030938,0.019502,0.040230,0.021395,0.027768,0.025306,0.034317,0.025328,...,0.051983,0.044183,0.021496,0.015634,0.019697,0.035357,0.021939,0.023071,0.029765,0.031301
5,complete,0.000025,0.000059,0.000138,0.000000,0.000000,0.000086,0.000084,0.000094,0.000041,...,0.000055,0.000017,0.000149,0.000000,0.000050,0.000102,0.000042,0.000063,0.000043,0.000056
6,works,0.000069,0.000355,0.000192,0.004970,0.001119,0.000082,0.000449,0.000793,0.000147,...,0.000935,0.000588,0.000433,0.000315,0.000269,0.000512,0.001353,0.000394,0.000245,0.000139
7,william,0.000092,0.000000,0.000000,0.000000,0.000170,0.000000,0.000000,0.000000,0.000004,...,0.000330,0.000034,0.000000,0.000131,0.000000,0.000320,0.000000,0.000027,0.000000,0.000000
8,shakespeare,0.000009,0.000000,0.000015,0.000000,0.000034,0.000006,0.000000,0.000000,0.000004,...,0.000000,0.000000,0.000000,0.000018,0.000000,0.000013,0.000000,0.000000,0.000000,0.000014
9,this,0.007402,0.003393,0.000409,0.011028,0.005968,0.005259,0.004938,0.006837,0.004799,...,0.005718,0.006252,0.004483,0.002550,0.002801,0.005711,0.004988,0.005239,0.004241,0.006449


In [14]:
query = str(input("Ingresa la palabra a buscar: "))
ranked_docs = rank_documents(query)
print(f"Documentos ordenados por similitud con la consulta '{query}':")
for doc, score in ranked_docs:
    print(f"{doc}: {score}")

Documentos ordenados por similitud con la consulta 'three':
pg2000.txt: 0.017308322793281407
pg20228.txt: 0.0321925809569759
pg47629.txt: 0.032485335101475325
pg10681.txt: 0.03634928853441162
pg1513.txt: 0.05972695105671366
pg100.txt: 0.061166906877525025
pg5740.txt: 0.06265589992226044
pg67979.txt: 0.06345821310642252
pg844.txt: 0.06722406469796459
pg45.txt: 0.06918912487394976
pg2641.txt: 0.07039266512721219
pg2554.txt: 0.07062984196153223
pg67098.txt: 0.07093867121163172
pg21700.txt: 0.07107018443921771
pg16389.txt: 0.0713104933301948
pg2542.txt: 0.07164595000031555
pg145.txt: 0.07189486303604609
pg600.txt: 0.0724668247590083
pg1260.txt: 0.07273313031698485
pg3825.txt: 0.07276164759819782
pg768.txt: 0.07282193858575037
pg1342.txt: 0.07301142361266386
pg28054.txt: 0.07321805346129127
pg64317.txt: 0.07354189924434201
pg16.txt: 0.07370620644955433
pg8800.txt: 0.07386188461628561
pg37106.txt: 0.07405278595828135
pg514.txt: 0.07405361305566266
pg394.txt: 0.07415471388384229
pg30254.txt: 

### Step 3: Binary Independence Model (BIM)

Task: Implement a basic Binary Independence Model to rank documents.

Requirements:
* _Binary Representation:_ Represent the corpus and the query in binary vectors (1 if the term is present, 0 otherwise).
* _Probability Estimation:_ Assume arbitrary probabilities for the presence of each term in relevant and non-relevant documents.
* _Relevance Scoring:_ Calculate the relevance score for each document based on the product of probabilities for terms present in the query.
* _Ranking:_ Rank the documents based on their relevance scores from highest to lowest.